In [1]:
! pip install python-docx

In [2]:
import os
import docx
import re
import sqlite3
import pandas as pd

In [3]:
interviews_paths = []

In [4]:
path = r'D:\Project Python 2022'
for root, directories, file in os.walk(path):
    for file in file:
        if(file.endswith(".docx")):
            interviews_paths.append(os.path.join(root,file))

In [5]:
print(interviews_paths[1])

D:\Project Python 2022\2021-06-25_Пряжа_Леонид Алексеевич_Недайхлеб, Неминова, Смирнова.docx


**Создание базы данных**

In [6]:
! pip install ipython-sql

In [7]:
%load_ext sql
%sql sqlite://
    
con = sqlite3.connect('karelia_database_project.db')  # подключение
cur = con.cursor() 

Таблица реплик

In [8]:
cur.execute("""
CREATE TABLE IF NOT EXISTS utterances (
    utterance_id  INTEGER PRIMARY KEY AUTOINCREMENT,
    speaker_id INT,
    speaker_role TEXT,
    interwiev_id INT,
    text TEXT, 
    utterance_number INT
)
""")

Таблица интервью

In [9]:
cur.execute("""
CREATE TABLE  IF NOT EXISTS interviews (
    interview_id INTEGER PRIMARY KEY AUTOINCREMENT,
    number_in_table INT,
    informants TEXT, 
    interwievers TEXT,
    place TEXT,
    day INT,
    month INT,
    year INT
)
""")

Таблица говорящих (информантов и интервьюеров)

In [10]:
cur.execute("""
CREATE TABLE  IF NOT EXISTS speaker (
    speaker_id  INTEGER PRIMARY KEY AUTOINCREMENT, 
    speaker_role TEXT,
    gender TEXT,
    birth_year INT
)
""")

Таблица с настоящими именами говорящих

In [11]:
cur.execute("""
CREATE TABLE IF NOT EXISTS speaker_names (
    speaker_id INTEGER PRIMARY KEY AUTOINCREMENT, 
    speaker_role TEXT,
    real_name TEXT
)
""")

In [12]:
id_search = """
SELECT speaker_id FROM speaker_names WHERE real_name LIKE ?
"""

In [13]:
interviewer_query = """
SELECT real_name from speaker_names
"""

In [14]:
con.commit()

Все регулярки

In [15]:
#метаинформация интервью
reg_number = re.compile("Номер интервью в базе.*(.[\d])")
reg_place = re.compile("Место\:\s?(.*)")
reg_data = re.compile("Дата\:\s?(.[\d.]*)")
reg_inf_names = re.compile("_?.*_(.*)_.*\.docx")
reg_int = re.compile("Инт.*—\s?(.*)")

#информант и интервьюер
reg_inf = re.compile("Инф.*—\s?(.*)")
reg_gender = re.compile("(\w{1})\,")
reg_birth_year = re.compile("\d{4}")
reg_int = re.compile("Инт.*—\s?(.*)")
reg_role_int = re.compile("(Инт.*)\s—\s?.*")
reg_role_inf = re.compile("(Инф.*)\s—\s?.*")

#отрывки
reg_line = re.compile("Ин..?:.*")
reg_speaker = re.compile("(Ин..?):.*")
reg_saying = re.compile("Ин..?:\s?(.*)")

**Парсер + запись в базу данных**

In [16]:
i = 1

for path in interviews_paths:
    
    #достаём текст
    doc = docx.Document(path)
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)
    text = '\n'.join(text)
     
    speakers_di = {}
    
    number = re.findall(reg_number, text)[0]
    place = re.findall(reg_place, text)[0]
    
   #парсер даты
    data = re.findall(reg_data, text)[0]
    day = re.findall(r"(.*)\..*\..*", data)[0]
    month = re.findall(r".*\.(.*)\..*", data)[0]
    year = re.findall(r".*\..*\.(.*)", data)[0]
    
    real_names = re.findall(reg_inf_names, path)[0]
    if ',' in real_names:
        real_names = real_names.split(', ')
    n = 0
    
    #парсер информантов
    informants = re.findall(reg_inf, text)
    for informant in informants:
        if len(re.findall(reg_gender, informant)) > 0:
            gender = re.findall(reg_gender, informant)[0]
        if len(re.findall(reg_birth_year, informant)) > 0:
            birth_year = re.findall(reg_birth_year, informant)[0]
        speaker_role = 'informant'
        data_informant = (speaker_role, gender, birth_year)
        cur.executemany("INSERT INTO speaker VALUES (NULL, ?, ?, ?)", (data_informant,))
        if type(real_names) == list:
            data_informant_real_name = [speaker_role, real_names[n]]
        elif type(real_names) == str:
            data_informant_real_name = [speaker_role, real_names]
        cur.executemany("INSERT INTO speaker_names VALUES (NULL, ?, ?)", (data_informant_real_name,))
        role = re.findall(reg_role_inf, text)[n]
        if type(real_names) == list:
            speakers_di[role] = real_names[n]
        elif type(real_names) == str:
            speakers_di[role] = real_names
        n += 1
    
    interviewers = re.findall(reg_int, text)
    m = 0
    for interviewer in interviewers:
        cur.execute(interviewer_query)
        result_set = cur.fetchall()
        li = []
        for row in result_set:
            li.append(row[0])
        if interviewer not in li:
            speaker_role = 'interviewer'
            data_interviewer = [speaker_role]
            cur.executemany("INSERT INTO speaker VALUES (NULL, ?, NULL, NULL)", (data_interviewer,))
            data_interviewer_real_name = [speaker_role, interviewers[m]]
            cur.executemany("INSERT INTO speaker_names VALUES (NULL, ?, ?)", (data_interviewer_real_name,))
        role = re.findall(reg_role_int, text)[m]
        speakers_di[role] = interviewers[m]
        m += 1
    #код поиска интервьюера в таблице интервьюеров
    
   # speakers_di #создать словарь спикеров интервью
    if len(interviewers) > 1:
        interviewers = '; '.join(interviewers)
    else:
        interviewers = interviewers[0]
    if len(informants) > 1:
        informants = '; '.join(informants)
    else:
        informants = informants[0]
    data1 = [number, informants, interviewers, place, day, month, year]
    cur.executemany("INSERT INTO interviews VALUES (NULL, ?, ?, ?, ?, ?, ?, ?)", (data1,))
    interview_id = i
    lines = re.findall(reg_line, text)
    utterance_number = 1
    for line in lines:
        speaker = re.findall(reg_speaker, line)
        saying = re.findall(reg_saying, line)[0]
        role = speaker[0]
        name = speakers_di[role]
        cur.execute(id_search, (name,))
        speaker_id = cur.fetchone()[0]
        if 'Инф' in speaker[0]:
            speaker_role = 'informant'
        else:
            speaker_role = 'interviewer'
        data3 = [speaker_id, speaker_role, interview_id, saying, utterance_number]
        cur.executemany("INSERT INTO utterances VALUES (NULL, ?, ?, ?, ?, ?)", (data3,))
        utterance_number += 1
    con.commit()
    i += 1

In [17]:
con.commit()